# Tutorial

In this tutorial, we will give a brief introduction on the quantization and pruning techniques upon which QSPARSE is built. Using our library, we guide you through the training of a [MNIST](http://yann.lecun.com/exdb/mnist/) image classification neural network, whose both weights and activations are fully quantized and pruned to a given sparsity.

> If you are already familiar with quantization and pruning methods and want to learn the programming syntax, please fast forward to [Building Network with QSPARSE](#building-network-with-qsparse).

## Preliminaries

Quantization and pruning are core techniques used to reduce the inference costs of deep neural networks and have been  studied extensively. Approaches to quantization are often divided into two categories: 

1. Post-training quantization
2. Quantization aware training

The former applies quantization after a network has been trained, and the latter quantizes the network during training and thereby reduces the quantization error throughout training process and usually yields superior performance. 

Pruning techniques are often divided into unstructured or structured approaches which define if and how to impose a pre-defined topology, e.g. channel-wise pruning. 

Here, we focus on applying quantization and unstructured pruning during training.

<figure> 
  <img src="../docs/assets/network_diagram-p1.svg" />
  <figcaption>Conceptual diagram of the computational graph of a network whose weights and activations are quantized and pruned using QSPARSE.</figcaption>
</figure>


In QSPARSE, we implement the quantization and pruning as independent operators, which can be applied on both weights and activations, as demonstrated in the figure above.

### Uniform Quantization

We denote the uniform quantization operation as $Q_u(\mathbf{x}, d)$, where $\mathbf{x}$ denotes the input to the operator  (i.e. weights or activations), $N$ denotes the total number of bits used to represent weights and activations, and $d$ denotes the number of bits used to represent the fractional bits the right of the decimal.

$$
Q_u(\mathbf{x}, d) = \text{clip}(\lfloor\mathbf{x} \times 2^{d}\rfloor, -2^{N-1}, 2^{N-1}-1) / 2^d
$$

Straight-through estimator (STE) is applied to calculate gradients in the backward computation.

$$
\frac{\partial Loss}{\partial \mathbf{x}} = \text{clip}(\frac{\partial Loss}{\partial Q_u(\mathbf{x}, d)}, -2^{N-d-1}, 2^{N-d-1} - 2^{-d})
$$

However, STE is known to be sensitive to weight initialization, therefore, we design the quantization operator as $\text{Quantize}$ in the following. Starting with the original full-precision network, we delay the quantization of the network to later training stages, and calculate the optimal decimal bits $d^*$ by minimizing the quantization error after a given number of update steps $t_q$.

$$
\text{Quantize}(\mathbf{x}_t)  = \begin{cases} 
    \mathbf{x}_t & t < t_q \\
    Q_u(\mathbf{x}_t, d^*)  &    t \ge t_q   \\
    \end{cases} 
$$

$$
d^* = \arg \min_{d} \Vert Q_u(\mathbf{x}_{t_q}, d) - \mathbf{x}_{t_q} \Vert^2
$$


### Magnitude-based Unstructured Pruning

We denote the unstructured pruning operator $\textbf{Prune}(\mathbf{x}, s)$ as element-wise multiplication between $\mathbf{x}$ and $\mathbf{M}_{\mathbf{x},s}$, where $\mathbf{x}$ denotes the input to the operator (i.e., weights or activations), $s$ denotes the target sparsity as measured by the percentage of zero-valued elements, and $\mathbf{M}_{\mathbf{x},s}$ denotes a binary mask.



Given that $(i,j)$ are the row and column indices, respectively, the binary mask $\mat{M}_{\mat{x},s}$ is calculated as belows, where the $\text{quantile}(\mathbf{x}, a)$ is the a-th quantile of $\mathbf{x}$.


### Training Schedule

often considered as independent problems.

## Building Network with QSPARSE

### Weight Quantization and Pruning

### Feature Quantization and Pruning

### Train a Network with Both Weight and Feature Quantized and Pruned


## Summary